In [1]:
# code for loading the format for the notebook
import os

# path : store the current path to convert back to it later
path = os.getcwd()
os.chdir('../notebook_format')
from formats import load_style
load_style()

In [2]:
os.chdir(path)
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential 
from keras.optimizers import SGD, RMSprop
from keras.layers.core import Dense, Dropout, Activation

Using Theano backend.


# Keras Basics

Basic Keras API to build a simple multi-layer neural network.

In [3]:
nb_classes = 10
num_features = 784

# load the dataset and some preprocessing step that can be skipped
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape( 60000, num_features )
X_test = X_test.reshape( 10000, num_features )
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [4]:
# convert class vectors to binary class matrices (one-hot encoding)
# note: you HAVE to to this step
Y_train = np_utils.to_categorical( y_train, nb_classes )
Y_test  = np_utils.to_categorical( y_test , nb_classes )

Basics of trainining a model:

you use `Sequential` to build models in keras, which basically represents we're going to later use the `.add()` method to stack layers together in sequence to build up our network.

- You start with `Dense` (fully-connected layers), where you specify how many nodes you wish to have for the layer. Since the first layer that we're going to add is the input layer, we have to make sure that the `input_dim` parameter matches the number of features (columns) in the training set. Then after the first layer, you don't need to specify the size of the input anymore.
- Then you specify the `Activation` function for that layer, and add a `Dropout` layer if you wish.
- For the last `Dense` and `Activation` layer you need to specify the number of class as the output and softmax to tell it to output the predicted class's probability.

In [5]:
# train a model:
model = Sequential()
model.add(Dense( 512, input_dim = (num_features,) ))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [6]:
# you can check the summary to check the number of parameters
model.summary()

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
dense_1 (Dense)                    (None, 512)         401920      dense_input_1[0][0]              
____________________________________________________________________________________________________
activation_1 (Activation)          (None, 512)         0           dense_1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)                (None, 512)         0           activation_1[0][0]               
____________________________________________________________________________________________________
dense_2 (Dense)                    (None, 512)         262656      dropout_1[0][0]                  
___________________________________________________________________________________________

Once your model looks good, configure its learning process with `.compile()`, where you need to specify which `optimizer` to use, and the `loss` function ( `categorical_crossentropy` is the typical one for multi-class classification) and the `metrics` to track. 

Finally, `.fit()` the model by passing in the training, validation set, the number of epochs and batch size.

In [7]:
model.compile( loss = 'categorical_crossentropy', optimizer = RMSprop(), metrics = ['accuracy'] )

nb_epoch = 10
batch_size = 128 # you typically specify this number to be power of 2
history = model.fit(
    X_train, 
    Y_train,
    batch_size = batch_size, 
    nb_epoch = nb_epoch,
    verbose = 1, # set it to 0 if you do not want to have progess bars
    validation_data = ( X_test, Y_test )
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 6s - loss: 0.2765 - acc: 0.9159 - val_loss: 0.1490 - val_acc: 0.9521
Epoch 2/10
60000/60000 [==============================] - 6s - loss: 0.1117 - acc: 0.9655 - val_loss: 0.0825 - val_acc: 0.9746
Epoch 3/10
60000/60000 [==============================] - 6s - loss: 0.0812 - acc: 0.9749 - val_loss: 0.0743 - val_acc: 0.9760
Epoch 4/10
60000/60000 [==============================] - 7s - loss: 0.0607 - acc: 0.9809 - val_loss: 0.0713 - val_acc: 0.9781
Epoch 5/10
60000/60000 [==============================] - 7s - loss: 0.0495 - acc: 0.9846 - val_loss: 0.0613 - val_acc: 0.9828
Epoch 6/10
60000/60000 [==============================] - 6s - loss: 0.0414 - acc: 0.9868 - val_loss: 0.0666 - val_acc: 0.9805
Epoch 7/10
60000/60000 [==============================] - 6s - loss: 0.0335 - acc: 0.9887 - val_loss: 0.0672 - val_acc: 0.9817
Epoch 8/10
60000/60000 [==============================] - 7s 

In [8]:
# history attribute stores the training and validation score and loss
history.history

{'acc': [0.9159333333651225,
  0.96548333330154423,
  0.97486666663487753,
  0.98091666666666666,
  0.98456666669845583,
  0.98678333336512247,
  0.98866666669845582,
  0.99084999996821088,
  0.99171666663487756,
  0.9922333333333333],
 'loss': [0.27654785337448118,
  0.11169736497203508,
  0.081167347753047944,
  0.060718721484144526,
  0.049504599696397783,
  0.041398391198615235,
  0.033502732425183056,
  0.029177681448310615,
  0.025318305787319938,
  0.023126852064952255],
 'val_acc': [0.95209999999999995,
  0.97460000000000002,
  0.97599999999999998,
  0.97809999999999997,
  0.98280000000000001,
  0.98050000000000004,
  0.98170000000000002,
  0.9829,
  0.98180000000000001,
  0.98509999999999998],
 'val_loss': [0.14897242912650108,
  0.08250118066966533,
  0.074268012980371709,
  0.071295447157695896,
  0.061288997134193779,
  0.06658970338082873,
  0.067245332123816476,
  0.062542211229892564,
  0.072774906067081604,
  0.057179510010950618]}

In [9]:
# .evaluate gives the score and loss for the dataset,
# here the result matches the validation set's history above
score = model.evaluate( X_test, Y_test, verbose = 0 )
score

[0.057179510334458249, 0.98509999999999998]

In [10]:
# stores the weight of the model,
# it's a list, note that the length is 6 because we have 3 dense layer
# and each one has it's associated bias term
weights = model.get_weights()
print( len(weights) )

# W1 should have 784, 512 for the 784
# feauture column and the 512 the number 
# of dense nodes that we've specified
W1, b1, W2, b2, W3, b3 = weights
print(W1.shape)
print(b1.shape)

6
(784, 512)
(512,)


In [11]:
# predict the accuracy
y_pred = model.predict_classes( X_test, verbose = 0 )
accuracy = np.sum( y_test == y_pred ) / X_test.shape[0]
print( 'valid accuracy: %.2f%%' % ( accuracy * 100 ) )

valid accuracy: 98.51%


## Saving and loading the models

It is not recommended to use pickle or cPickle to save a Keras model.

In [16]:
# 1. to_json saves the model's architecture
# 2. save_weights, save the weights learned by the model to a .h5 file
json_string = model.to_json()
open('my_model_architecture.json', 'w').write(json_string)
model.save_weights('my_model_weights.h5')

# reload the model, weights that corresponds to the same architecture
from keras.models import model_from_json
model1 = model_from_json( open('my_model_architecture.json').read() )
model1.load_weights('my_model_weights.h5')

In [19]:
# note that you still need to compile the model again,
# this time use another kind of optimizer, note that you can also
# pass in a string to it, e.g. 'sgd' and the default parameter will be
# used. Here the different optimizer doesn't affect anything, since
# we're only compiling it
sgd = SGD( lr = 0.001, decay = 1e-7, momentum = .9 )
model1.compile( loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'] )

In [20]:
# testing: predict the accuracy using the loaded model
y_pred = model1.predict_classes( X_test, verbose = 0 )
accuracy = np.sum( y_test == y_pred ) / X_test.shape[0]
print( 'valid accuracy: %.2f%%' % ( accuracy * 100 ) )

valid accuracy: 98.51%


## Reference

- [Keras Documentation](http://keras.io/) 
- [mnist_mlp example](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py)
- [Saving Keras Model](http://keras.io/getting-started/faq/#how-can-i-save-a-keras-model)